## This notebook identifies **characteristic peaks** for **specific dihydroxy bile acid compounds**
(original notebook content: v_boxplot_all_di_M+H_for_manuscript.ipynb)

---

- Specific dihydroxy bile acids were determined by substructures defined by two hydroxylation events: C3 and C6; C3 and C7; and C3 and C12 described in **substructure_search_identify_di_BA.ipynb**
- **output is a boxplot (and corresponding dataframes)** describing the characteristic peaks and their stats

---

### About the process
The process for determining consistent patterns of fragmentation is as follows:

1. MS/MS peaks are grouped in 0.01 M/Z bins to understand fragmentation patterns with higher peak precision
2. Peaks attributed to noise are removed by isolating peaks above a normalized intensity of 0.05
3. Minimum percent occurrence for peaks of interest (after removing noise) was specified to occur in at least 20% of the grouped spectra

Final output summarizes the MS/MS peak intensity distribution represented by those peaks occurring at least 20% of the time.

---
## Notebook organization

### Section 1: Reading input data
- MS/MS peak data
- dihydroxy-BA specific metadata in GNPS Library

### Section 2: Matching peak data with metadata

### Section 3: Investigate peak fragmentation
- Make 0.01 M/Z bins
- Define minimum intensity and percent occurrence parameters
- Identify peaks that satisfy parameters

### Section 4: Plotting

## Input files needed for the Notebook
1. MS/MS peak data
2. Dataframe output of dihydroxy-BA metadata in GNPS Library from **substructure_search_identify_di_BA.ipynb**

In [1]:
import pandas as pd
import plotly
import plotly.express as px

### Section 1: Read input data

#### Read peak data
- from v_get_peaks_files.ipynb

(Neededed to break into 5 parts due to file size)

In [2]:
all_file_peaks_part_1 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_1.gzip')

In [3]:
all_file_peaks_part_2 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_2.gzip')

In [4]:
all_file_peaks_part_3 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_3.gzip')

In [5]:
all_file_peaks_part_4 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_4.gzip')

In [6]:
all_file_peaks_part_5 = pd.read_parquet('/home/jovyan/work/notebooks/outputs/all_file_peaks_part_5.gzip')

#### Read dihydroxy bile acid metadata from GNPS Library
- from substructure_search_identify_di_BA.ipynb

In [7]:
di_table = pd.read_csv('/home/jovyan/work/notebooks/outputs/library_df_dihydroxy_BA_only_m+h_updated_matches.csv',sep=',', index_col='spectrum_id')

In [8]:
# list of spectrum_id
di_table_ID = di_table.index.to_list()

### Section 2: Matching peak data with metadata
- Need to identify MS/MS peak data for spectra identified as dihydroxy bile acids in GNPS Library by substructure search

In [9]:
all_file_peaks_part_1_di = all_file_peaks_part_1[all_file_peaks_part_1.index.isin(di_table_ID)]

In [10]:
all_file_peaks_part_2_di = all_file_peaks_part_2[all_file_peaks_part_2.index.isin(di_table_ID)]

In [11]:
all_file_peaks_part_3_di = all_file_peaks_part_3[all_file_peaks_part_3.index.isin(di_table_ID)]

In [12]:
all_file_peaks_part_4_di = all_file_peaks_part_4[all_file_peaks_part_4.index.isin(di_table_ID)]

In [13]:
all_file_peaks_part_5_di = all_file_peaks_part_5[all_file_peaks_part_5.index.isin(di_table_ID)]

In [14]:
# Combine individual dataframes to make complete dataframe of all peak data associated with Di-BA
all_file_peaks_di = pd.concat([all_file_peaks_part_1_di, all_file_peaks_part_2_di, all_file_peaks_part_3_di,
                                      all_file_peaks_part_4_di, all_file_peaks_part_5_di], axis=0)

In [15]:
all_file_peaks_di

,level_0,index,i,i_norm,i_tic_norm,mz,mz_nl,precmz
scan,,,,,,,,
CCMSLIB00010012258,1927,0,292.0,0.064673,0.013038,145.099136,558.430864,703.530
CCMSLIB00010012258,1928,1,90.0,0.019934,0.004019,146.101257,557.428743,703.530
CCMSLIB00010012258,1929,2,341.0,0.075526,0.015226,147.116302,556.413698,703.530
CCMSLIB00010012258,1930,3,509.0,0.112735,0.022727,149.131912,554.398088,703.530
CCMSLIB00010012258,1931,4,116.0,0.025692,0.005179,150.134186,553.395814,703.530
...,...,...,...,...,...,...,...,...
CCMSLIB00000222745,41152,2,303.0,0.303303,0.110544,343.264099,48.020901,391.285
CCMSLIB00000222745,41153,3,726.0,0.726727,0.264867,345.280487,46.004513,391.285
CCMSLIB00000222745,41154,4,177.0,0.177177,0.064575,347.296692,43.988308,391.285


In [16]:
# check that all spectrum_id are accounted for
len(all_file_peaks_di.index.unique().to_list())

690

### Section 3: Investigate peak fragmentation

In [17]:
# rename combined dataframe
peak_df = all_file_peaks_di

In [18]:
# Identifying small bins of MZ values
peak_df['mz_binned_small'] = peak_df['mz'].round(decimals = 2)
unique_mz_binned = peak_df['mz_binned_small'].unique()

In [19]:
peak_df.head(5)

,level_0,index,i,i_norm,i_tic_norm,mz,mz_nl,precmz,mz_binned_small
scan,,,,,,,,,
CCMSLIB00010012258,1927,0,292.0,0.064673,0.013038,145.099136,558.430864,703.53,145.10
CCMSLIB00010012258,1928,1,90.0,0.019934,0.004019,146.101257,557.428743,703.53,146.10
CCMSLIB00010012258,1929,2,341.0,0.075526,0.015226,147.116302,556.413698,703.53,147.12
CCMSLIB00010012258,1930,3,509.0,0.112735,0.022727,149.131912,554.398088,703.53,149.13
CCMSLIB00010012258,1931,4,116.0,0.025692,0.005179,150.134186,553.395814,703.53,150.13


In [20]:
# Defining parameters --> can be modified by user

intensitynormmin  = 0.05
percentoccurmin = 20

In [21]:
# identifying peaks that satisfy minimum normalized intensity parameter

filtered_peak_df_i_norm = peak_df[peak_df["i_norm"] >= intensitynormmin]

In [22]:
# For counting percent occurrence of peaks above miniumum intensity
occurs_above_intensitynormmin = {}

# Total number of spectral IDs
total_ids = len(peak_df.index.unique())

for peak in unique_mz_binned:
    mz_df_above_intensitynormmin = filtered_peak_df_i_norm.loc[(filtered_peak_df_i_norm['mz_binned_small'] == peak)]

    # Number of spectra where peak occurs above miniumum intensity
    peak_occurs_above_intensitynormmin = len(mz_df_above_intensitynormmin)

    if peak_occurs_above_intensitynormmin/total_ids >= (percentoccurmin/100):
        occurs_above_intensitynormmin[peak] = peak_occurs_above_intensitynormmin/total_ids

In [23]:
# Filtering to only include peaks that are present in at least 20% of the scans
filtered_peak_df = filtered_peak_df_i_norm[filtered_peak_df_i_norm["mz_binned_small"].isin(occurs_above_intensitynormmin.keys())]

In [24]:
filtered_peak_df

,level_0,index,i,i_norm,i_tic_norm,mz,mz_nl,precmz,mz_binned_small
scan,,,,,,,,,
CCMSLIB00010012258,1927,0,292.0,0.064673,0.013038,145.099136,558.430864,703.530,145.10
CCMSLIB00010012258,1929,2,341.0,0.075526,0.015226,147.116302,556.413698,703.530,147.12
CCMSLIB00010012258,1930,3,509.0,0.112735,0.022727,149.131912,554.398088,703.530,149.13
CCMSLIB00010012258,1935,8,259.0,0.057364,0.011565,159.116196,544.413804,703.530,159.12
CCMSLIB00010012258,1938,11,507.0,0.112292,0.022638,161.130783,542.399217,703.530,161.13
...,...,...,...,...,...,...,...,...,...
CCMSLIB00005738654,289953,576,1102.0,0.192725,0.009282,322.260406,178.043594,500.304,322.26
CCMSLIB00005738654,289964,587,5102.0,0.892270,0.042975,339.268097,161.035903,500.304,339.27
CCMSLIB00005738654,289966,589,1452.0,0.253935,0.012230,340.273285,160.030715,500.304,340.27


In [25]:
# reshaping and renaming
peak_ratio_df = pd.DataFrame.from_dict(occurs_above_intensitynormmin, orient='index')
peak_ratio_df.index.name = 'mz_binned_small'
peak_ratio_df = peak_ratio_df.rename(columns={0: "percent_occurrence"})

In [26]:
# visualize peaks by descending percent occurrence
peak_ratio_df.sort_values(['percent_occurrence'], ascending=False).head(5)

,percent_occurrence
mz_binned_small,
339.27,0.604348
321.26,0.582609
215.18,0.552174
161.13,0.533333
175.15,0.444928


### Section 4: Plotting

In [45]:
with open('/home/jovyan/work/notebooks/outputs/output_box_peak_all_di_M+H_df.html', 'w') as f:

        ax = px.box(filtered_peak_df, x='mz_binned_small',y = 'i_norm')

        # Use hidden line below to graph ratio of MISSING peaks
        #ax.add_bar(x=peak_ratio_df.index, y=-peak_ratio_df["ratio of missing peaks"], name = "ratio of spectra with missing peaks")
        
        ax.add_bar(x=peak_ratio_df.index, y=-peak_ratio_df["percent_occurrence"],
                   name = "ratio of spectra where at least " + str(percentoccurmin)+"% contain peak",
                   width = 1)
        
        ax.update_layout(title_text="MS/MS Peak Intensity Distribution for " +"All Dihydroxy BA")
        
        f.write(ax.to_html(full_html=False, include_plotlyjs='cdn'))

### Save files

In [46]:
filtered_peak_df.reset_index().to_csv(
    '/home/jovyan/work/notebooks/bile_acid_notebooks/files_for_boxplot/filtered_peak_df_BA_di_3_7_M+H.csv', sep=',', index=False)

In [47]:
peak_ratio_df.reset_index().to_csv(
    '/home/jovyan/work/notebooks/bile_acid_notebooks/files_for_boxplot/peak_ratio_df_BA_di_3_7_M+H.csv', sep=',', index=False)